In [1]:
FILE_PATH = ''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        FILE_PATH = os.path.join(dirname, filename)

In [2]:
import tensorflow as tf

# Check if TensorFlow is using GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


In [3]:
# Method 2: Using open and close explicitly
file = open(FILE_PATH, 'r')
content = file.read()
file.close()

In [4]:
#Remove punctuations
import re
_content = re.sub(r'[^\w\s]', '', content)
content_clean = _content.lower()[:1000000]

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([content_clean])
unique_cnt = len(tokenizer.word_index)
print("Total unique words in the document is ", unique_cnt)

Total unique words in the document is  4606


In [7]:
total_sentences = content_clean.split('\n')
print("Total sentences are ",len(total_sentences))

Total sentences are  951


In [8]:
input_sequences = []
for sentence in total_sentences:
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1,len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i+1])

In [9]:
max_len = max([len(x) for x in input_sequences])
print("Size of the longest sentence is ",max_len)

Size of the longest sentence is  351


In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [11]:
padded_input_sequences

array([[  0,   0,   0, ...,   0,   1, 114],
       [  0,   0,   0, ...,   1, 114,  93],
       [  0,   0,   0, ...,   0, 223, 400],
       ...,
       [  0,   0,   0, ..., 326,   5, 244],
       [  0,   0,   0, ...,   5, 244,  96],
       [  0,   0,   0, ..., 244,  96,   2]], dtype=int32)

In [12]:
X = padded_input_sequences[:,:-1]

In [13]:
X.shape

(37641, 350)

In [14]:
y = padded_input_sequences[:,-1]

In [15]:
y.shape

(37641,)

In [16]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=unique_cnt+1)

In [17]:
y.shape

(37641, 4607)

In [18]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

In [19]:
X.shape

(37641, 350)

In [20]:
y.shape

(37641, 4607)

In [21]:
model_two = Sequential()
model_two.add(Embedding(unique_cnt+1, 128, input_shape=(max_len,)))
model_two.add(LSTM(256))
model_two.add(Dense(unique_cnt+1, activation='softmax'))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [22]:
model_two.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [23]:
model_two.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 351, 128)       │       589,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 256)            │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4607)           │     1,183,999 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,167,935 (8.27 MB)

 Trainable params: 2,167,935 (8.27 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model_two.fit(X,y,epochs=42)

Epoch 1/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 28s 22ms/step - accuracy: 0.0769 - loss: 6.5858
Epoch 2/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - accuracy: 0.1135 - loss: 5.6748
Epoch 3/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 27s 23ms/step - accuracy: 0.1431 - loss: 5.1371
Epoch 4/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - accuracy: 0.1666 - loss: 4.6839
Epoch 5/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - accuracy: 0.1914 - loss: 4.2449
Epoch 6/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - accuracy: 0.2374 - loss: 3.7908
Epoch 7/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - accuracy: 0.3014 - loss: 3.3137
Epoch 8/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - accuracy: 0.3721 - loss: 2.8875
Epoch 9/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - accuracy: 0.4486 - loss: 2.5339
Epoch 10/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - accuracy: 0.5159 - loss: 2.1860
Epoch 11/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - accuracy: 0.5830 - loss: 1.8902
Epoch 12

In [35]:
import time
import numpy as np
# text = "what is the fee"
texts = ['Hello I am','I dont like it here because','I hate','Trees give us','Chicken is very']
for text in texts:
    token_text = tokenizer.texts_to_sequences([text])[0]
    padded_token_text = pad_sequences([token_text], maxlen=max_len, padding='pre')
    pos = np.argmax(model_two.predict(padded_token_text))
    for word,index in tokenizer.word_index.items():
        if index == pos:
            text = text + " " + word
            print(text)
            time.sleep(2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Hello I am made
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
I dont like it here because i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
I hate people
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Trees give us my
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chicken is very silly


In [36]:
def predict(text):
    token_text = tokenizer.texts_to_sequences([text])[0]
    padded_token_text = pad_sequences([token_text], maxlen=max_len, padding='pre')
    pos = np.argmax(model_two.predict(padded_token_text))
    for word,index in tokenizer.word_index.items():
        if index == pos:
            text = text + " " + word
            print(text)

In [39]:
predict("Gambhir is very angry because")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Gambhir is very angry because i


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
MS is very silly
